# Chapter 2 Solutions

In [4]:
import numpy as np

##1. This problem concerns the ALOHA network model of Section 2.1. Feel free to use (but cite) computations already in the example.

In [316]:
class Constants():
    p = 0.4 # probability of sending as an active node
    q = 0.8 # probability of becoming active as an inactive node
    num_samples = 1000.0
    num_experiments = 1000000
    num_new_nodes = 2
        
class Sample():
    def __init__(self, ratio):
        self.ratio = ratio
        
    def generate(self):
        num_ones = int(self.ratio * Constants.num_samples)
        num_zeros = int((1 - self.ratio) * Constants.num_samples)
        return np.append(np.zeros(num_zeros), np.ones(num_ones))

    def event_happens(self):
        samples = self.generate()
        np.random.shuffle(samples)
        return samples[0] == 1

class InactiveNode():
    def maybe_become_active(self):
        if Sample(Constants.q).event_happens():
            return ActiveNode()
        else: 
            return InactiveNode()
        
    def is_sendable(self):
        return False
    
    def maybe_send(self, collision):
        return InactiveNode()
    
    def is_active(self):
        return False
    
    def is_inactive(self):
        return not self.is_active()



class ActiveNode():
    def __init__(self):
        self.sendable = False

    def maybe_become_active(self):
        return ActiveNode()
    
    def is_sendable(self):
        self.sendable = Sample(Constants.p).event_happens()
        return self.sendable
    
    def maybe_send(self, collision):
        if collision or not self.sendable:
            return ActiveNode()
        else:    
            return InactiveNode()

    def is_active(self):
        return True
    
    def is_inactive(self):
        return not self.is_active()


class Epoch():
    def __init__(self, nodes, num):
        self.nodes = nodes
        self.num = num

    def check_collisions(self):
        collision = True

        for node in self.nodes:
            collision = collision & node.is_sendable()
        self.collision = collision

    def maybe_send(self):
        new_nodes = []
        for node in self.nodes:
            new_nodes.append(node.maybe_send(self.collision))
        return new_nodes

    def maybe_become_active(self):
        new_nodes = []
        for node in self.nodes:
            new_nodes.append(node.maybe_become_active())
        self.nodes = new_nodes

    def num_nodes_active(self):
        count = 0
        for node in self.nodes:
            if node.is_active():
                count += 1
        return count

def simulate_aloha(criteria):
    

    def build_epoch(epoch=Epoch([ActiveNode(), ActiveNode()], 0)):
        epoch.maybe_become_active()
        epoch.check_collisions()
        
        return Epoch(epoch.maybe_send(), epoch.num + 1)
    
    def generate_initial_number_of_nodes():
        nodes = []
        for i in range(Constants.num_new_nodes):
            nodes.append(ActiveNode())
        return nodes
    
    def populate_active_count(num_new_epochs):
        epoch = Epoch(generate_initial_number_of_nodes(), 0)
        active_count = [epoch.num_nodes_active()]
        for i in range(num_new_epochs):
            epoch = build_epoch(epoch=epoch)
            active_count.append(epoch.num_nodes_active())
        return active_count
    
    counts_updater = CountsUpdater(criteria)

    for experiment in range(Constants.num_experiments):
        active_count = populate_active_count(Constants.num_new_nodes)
        counts_updater.update(active_count)

    # needs to divide by num times that the "given" section happens, if "given" exists
    if counts_updater.denominator == 0:
        return 0.0
    else:
        return counts_updater.numerator / counts_updater.denominator
        
    
class CountsUpdater():
    def __init__(self, criteria):
        self.criteria = criteria
        self.numerator = 0.0
        self.denominator = 0.0
    
    def update(self, active_count):
        if self.criteria.given(active_count):
            self.denominator += 1
        if self.criteria.given(active_count) and self.criteria.criteria(active_count):
            self.numerator += 1

class CriteriaWithCondition(object):
#     def __init__(self):
#         self.divisor_count = 0.0
#         self.numerator_count = 0.0
        
    def given(self, active_count):
        raise Exception('Implement me!')
        
    def criteria(self, active_count):
        raise Exception('Implement me!')

class CriteriaWithoutCondition():
#     def __init__(self):
#         self.divisor_count = Constants.num_experiments
#         self.numerator_count = 0.0
        
    def given(self, active_count):
        return True
    
    def criteria(self, active_count):
        raise Exception('Implement me!')




     
   
    

In [260]:
class CriteriaSanityCheck2Dot1(CriteriaWithoutCondition):
    def criteria(self, active_count):
        return active_count[1] == 2

# should be 0.52-ish
simulate_aloha(CriteriaSanityCheck2Dot1())

0.5191

In [261]:
class CriteriaSanityCheck2Dot16(CriteriaWithoutCondition):
    def criteria(self, active_count):
        return active_count[1] == 1  

# should be 0.48-ish
simulate_aloha(CriteriaSanityCheck2Dot16())

0.4796

In [262]:
class CriteriaSanityCheck2Dot17(CriteriaWithCondition):
    def given(self, active_count):
        return active_count[1] == 1
    
    def criteria(self, active_count):
        return active_count[2] == 2
    
# should be 0.41-ish
simulate_aloha(CriteriaSanityCheck2Dot17())

0.42521501992867633

In [263]:
class CriteriaSanityCheck2Dot18(CriteriaWithCondition):
    def given(self, active_count):
        return active_count[2] == 2
    
    def criteria(self, active_count):
        return active_count[1] == 1
    

# should be 0.43-ish
simulate_aloha(CriteriaSanityCheck2Dot18())

0.43349648861459883

In [264]:
class CriteriaSanityCheck2Dot19(CriteriaWithoutCondition):
    def criteria(self, active_count):
        return active_count[1] == 2 or active_count[2] == 2

# should be 0.72-ish
simulate_aloha(CriteriaSanityCheck2Dot19())

0.722

## (a) Calculate $P(X_{1} = 2 \text{ and } X_{2} = 1)$, for the same values of $p$ and $q$ in the examples.

\begin{equation}
  P(X_{1} = 2 \text{ and } X_{2} = 1) = P(X_{2} = 1 \mid X_{1} = 2) P(X_{1} = 2)
\end{equation}

Formally speaking:

\begin{equation}
\begin{split}
  P(X_{1} = 2) &= \sum_{i=0}^{2}{P(X_{1} = 2 \mid X_{0} = i) P(X_{0} = i)} \\
  &= P(X_{1} = 2 \mid X_{0} = 0) P(X_{0} = 0) \\
  &+  P(X_{1} = 2 \mid X_{0} = 1) P(X_{0} = 1) \\
  &+ P(X_{1} = 2 \mid X_{0} = 2) P(X_{0} = 2)
\end{split}
\end{equation}


However, since it was given that both nodes were active in the beginning (i.e.
$P(X_{0} = 2) = 1$), we know that $P(X_{0} = 0) = P(X_{0} = 1) = 0$. Thus, the
statement above simplifies to:

\begin{equation}
\begin{aligned}
  P(X_{1} = 2) &= P(X_{1} = 2 \mid X_{0} = 2)
\end{aligned}
\end{equation}

$P(X_{1} = 2 \mid X_{0} = 2)$ could only happen two ways: either both
active nodes send information (and hence create a collision), or both active
nodes don't send anything at all.

\begin{equation}
  \begin{aligned}
    P(X_{1} = 2) &= P(X_{1} = 2 \mid X_{0} = 2) \\
    &= P(C_{1} = 1 \text{ and } C_{2} = 1 \text{ or } C_{1} = 0 \text{ and } C_{2} = 0) \\
                 &= P(C_{1} = 1 \text{ and } C_{2} = 1) + P(C_{1} = 0 \text{ and } C_{2} = 0) \\
    &= p^2 + (1-p)^2 \\
    &= (0.4)^2 + (1-0.4)^2 \\
    &= 0.16 + 0.36 \\
    &= 0.52 \\
  \end{aligned}
\end{equation}

\begin{equation}
  \begin{aligned}
    P(X_{2} = 1 \mid X_{1} = 2) &= P(C_{1} = 1 \text{ and } C_{2} = 0 \text{ or } C_{1} = 0 \text{ and } C_{2} = 1) \\
    &= P(C_{1} = 1 \text{ and } C_{2} = 0) + P(C_{1} = 0 \text{ and } C_{2} =  1) \\
    &= p(1-p) + p(1-p) \\
    &= 2p(1-p) \\
    &= 2(0.4)(1-(0.4)) \\
    &= 0.48\\
  \end{aligned}
\end{equation}

Therefore:

\begin{equation}
  \begin{aligned}
    P(X_{1} = 2 \text{ and } X_{2} = 1) &= P(X_{2} = 1 \mid X_{1} = 2) P(X_{1} = 2) \\
    &= 0.48 \times 0.52 \\
    &= 0.2496
  \end{aligned}
\end{equation}




In [265]:
class CriteriaProblem1a(CriteriaWithoutCondition):
    def criteria(self, active_count):
        return active_count[1] == 2 and active_count[2] == 1

# should be 0.2496-ish
simulate_aloha(CriteriaProblem1a())

0.2497

## (a) Find $P(X_{2} = 0)$


\begin{equation}
  \begin{aligned}
    P(X_2 = 0) &= \sum_{i=0}^{2} P(X_2 = 0 \text{ and } X_1 = i) \\
    &= \sum_{i=0}^{2} P(X_2 = 0 \mid X_1 = i)P(X_1 = i) \\
    &= P(X_2 = 0 \mid X_1 = 0)P(X_1 = 0) \\
    &\quad + P(X_2 = 0 \mid X_1 = 1)P(X_1 = 1) \\
    &\quad + P(X_2 = 0 \mid X_1 = 2)P(X_1 = 2) \\
  \end{aligned}
\end{equation}

Let's start with $P(X_1 = 0)$. We know that it is equivalent to:

\begin{equation}
  \begin{aligned}
    P(X_1 = 0) &= P(X_1 = 0 \text{ and } X_0 = 2) \\
    &= P(X_1 = 0 \mid X_0 = 2)P(X_0 = 2) \\
    &= P(X_1 = 0 \mid X_0 = 2) \\
  \end{aligned}
\end{equation}

It is impossible for two active nodes to both become inactive for the next
epoch, so $P(X_1 = 0) = P(X_1 = 0 \mid X_0 = 2) = 0$.

Next we look at $P(X_1 = 1)$:

\begin{equation}
  \begin{aligned}
    P(X_1 = 1) &= P(X_1 = 1 \text{ and } X_0 = 2) \\
    &= P(X_1 = 1 \mid X_0 = 2)P(X_0 = 2) \\
    &= P(X_1 = 1 \mid X_0 = 2) \\
  \end{aligned}
\end{equation}

This could only happen two ways: either the first node sends and the second
node does not, or the second node sends and the first node does not. For active
nodes, let $S_i=k, i \in \{0,1\}$ and $k \in \{0,1\}$ where, for node $i$,
$S_i=0$ is the event of not sending and $S_i=1$ is the event of sending:

\begin{equation}
  \begin{aligned}
    P(X_1 = 1) &= P(X_1 = 1 \mid X_0 = 2) \\
    &= P(S_1 = 1 \text{ and } S_2 = 0 \text{ or } S_1 = 0 \text{ and } S_2 = 1) \\
    &= P(S_1 = 1 \text{ and } S_2 = 0) + P(S_1 = 0 \text{ and } S_2 = 1) \\
    &= p(1-p) + p(1-p) \\
    &= 2p(1-p) \\
    &= 2(0.4)(1-0.4) \\
    &= 0.48
  \end{aligned}
\end{equation}

Now let's look at $P(X_2=0 \mid X_1=1)$. The only way this could happen is when
the active node sends while the other node stays inactive:

\begin{equation}
  \begin{aligned}
    P(X_2=0 \mid X_1=1) &= p(1-q) \\
    &= (0.4)(1-(0.8)) \\
    &= 0.08
  \end{aligned}
\end{equation}

Now we consider $P(X_2=0 \mid X_1=2)$. Again it is impossible for two active
nodes to become inactive for the next epoch, so $P(X_2=0 \mid X_1=2) = 0$.

Therefore, $P(X_2=0)$ simplifies to the following:

\begin{equation}
  \begin{aligned}
    P(X_2=0) &= P(X_2 = 0 \mid X_1 = 0)P(X_1 = 0) \\
    &\quad + P(X_2 = 0 \mid X_1 = 1)P(X_1 = 1) \\
    &\quad + P(X_2 = 0 \mid X_1 = 2)P(X_1 = 2) \\
    &= P(X_2 = 0 \mid X_1 = 0) \times 0 \\
    &\quad + 0.08 \times 0.48 \\
    &\quad + 0 \times P(X_1 = 2) \\
    &= 0.0384
  \end{aligned}
\end{equation}





In [266]:
class CriteriaProblem1b(CriteriaWithoutCondition):
    def criteria(self, active_count):
        return active_count[2] == 0
    
# should be 0.0384-ish
simulate_aloha(CriteriaProblem1b())

0.0379

## (a) Find $P(X_{1} = 1 \mid X_{2} = 1)$

\begin{equation}
  \begin{aligned}
    P(X_{1} = 1 \mid X_{2} = 1) &= \frac{P(X_1=1 \text{ and } X_2=1)}{P(X_2 = 1)} \\
    &= \frac{P(X_2=1 \mid X_1=1)P(X_1=1) }{P(X_2 = 1)} \\
    &= \frac{P(X_2=1 \mid X_1=1)P(X_1=1) }{\sum_{i=0}^{2}P(X_2 = 1, X_1=i)} \\
    &= \frac{P(X_2=1 \mid X_1=1)P(X_1=1) }{\sum_{i=0}^{2}P(X_2 = 1 \mid X_1=i)P(X_1=i)} \\
  \end{aligned}
\end{equation}

The denominator expands to the following:

\begin{equation}
  \begin{aligned}
    \sum_{i=0}^{2}P(X_2 = 1 \mid X_1=i)P(X_1=i) &= P(X_2 = 1 \mid X_1=0)P(X_1=0) \\
    &\quad + P(X_2 = 1 \mid X_1=1)P(X_1=1) \\
    &\quad + P(X_2 = 1 \mid X_1=2)P(X_1=2) \\
  \end{aligned}
\end{equation}

We've already established that $P(X_1=0) = 0$, $P(X_1=1)=0.48$, $P(X_1=2) =
0.52$. Since, $P(X_1=0) = 0$, we know that $P(X_2=1 \mid X_1=0)$ cannot
happen. Thus, what we need to calculate are the following: $P(X_2=1
\mid X_1=1)$ and $P(X_2=1 \mid X_1=2)$.

Let's start with $P(X_2=1 \mid X_1=1)$. Three possibilities:

* Active node successfully sends; inactive node becomes active and does not send
* Active node does not attempt to send; inactive node does not become active
* Active node does not attempt to send; inactive node becomes active and successfully sends

\begin{equation}
  \begin{aligned}
    P(X_2=1 \mid X_1=1) &= (1-p)qp + (1-p)(1-q) + pq(1-p) \\
    &= (1-0.4)(0.8)(0.4) + (1-0.4)(1-0.8) + (0.4(0.8(1-0.4))) \\
    &= 0.192 + 0.12 + 0.192 \\
    &= 0.504
  \end{aligned}
\end{equation}

Next we look at $P(X_2=1 \mid X_1=2)$. The first active node sends a message
and becomes inactive while the second active node refrains from sending, or the
second active node sends a message and becomes inactive while the first active
node refrains.

\begin{equation}
  \begin{aligned}
    P(X_2=1 \mid X_1=2) &= P(A_1=1 \text{ and } A_2=0 \text{ or } A_1=0 \text{ and } A_2=1) \\
    &= P(A_1=1 \text{ and } A_2=0) + P(A_1=0 \text{ and } A_2=1) \\
    &= p(1-p) + (1-p)p \\
    &= 2p(1-p) \\
    &= 2(0.4)(1-0.4) \\
    &= 0.48
  \end{aligned}
\end{equation}

Therefore, $P(X_1=1 \mid X_2=1)$ reduces to the following:

\begin{equation}
  \begin{aligned}
    P(X_1=1 \mid X_2=1) &= \frac{P(X_2=1 \mid X_1=1)P(X_1=1) }{\sum_{i=0}^{2}P(X_2 = 1 \mid X_1=i)P(X_1=i)} \\
    &= \frac{0.504 \times 0.48}{0 \times 0 + 0.504 \times 0.48 + 0.48 \times 0.52}\\
    &= \frac{0.24192}{0 + 0.24192 + 0.2496} \\
    &= 0.4921875 \\
  \end{aligned}
\end{equation}




In [317]:
class CriteriaProblem1c(CriteriaWithCondition):
    def given(self, active_count):
        return active_count[2] == 1
    def criteria(self, active_count):
        return active_count[1] == 1
    
# should be 0.55-ish
simulate_aloha(CriteriaProblem1c())

0.49233776547005764

In [268]:
class CriteriaSanityCheckProblem1c_1(CriteriaWithCondition):
    def given(self, active_count):
        return active_count[1] == 1
    def criteria(self, active_count):
        return active_count[2] == 1
    
# should be 0.48-ish
simulate_aloha( CriteriaSanityCheckProblem1c_1())

0.5061957868649318

In [247]:
class CriteriaSanityCheckProblem1c_2(CriteriaWithoutCondition):
#     def given(self, active_count):
#         return active_count[1] == 1
    def criteria(self, active_count):
        return active_count[1] == 0
    
# should be 0
simulate_aloha(CriteriaSanityCheckProblem1c_2())

0.0

In [249]:
class CriteriaSanityCheckProblem1c_3(CriteriaWithoutCondition):
#     def given(self, active_count):
#         return active_count[1] == 1
    def criteria(self, active_count):
        return active_count[1] == 1
    
# should be 0.48
simulate_aloha(CriteriaSanityCheckProblem1c_3())

0.4827

In [255]:
class CriteriaSanityCheckProblem1c_4(CriteriaWithoutCondition):
#     def given(self, active_count):
#         return active_count[1] == 1
    def criteria(self, active_count):
        return active_count[1] == 2
    
# should be 0.52
simulate_aloha(CriteriaSanityCheckProblem1c_4())

0.5242

In [256]:
class CriteriaSanityCheckProblem1c_5(CriteriaWithCondition):
    def given(self, active_count):
        return active_count[1] == 0
    def criteria(self, active_count):
        return active_count[2] == 1
    
# should be 0
simulate_aloha(CriteriaSanityCheckProblem1c_5())

0.0

In [281]:
class CriteriaSanityCheckProblem1c_6(CriteriaWithCondition):
    def given(self, active_count):
        return active_count[1] == 2
    def criteria(self, active_count):
        return active_count[2] == 1
    
# should be 0.48
simulate_aloha(CriteriaSanityCheckProblem1c_6())

0.4727307766428984

In [300]:

class CriteriaSanityCheckProblem1c_7(CriteriaWithCondition):
    def given(self, active_count):
        return active_count[1] == 1
    def criteria(self, active_count):
        return active_count[2] == 1
    
# should be 0.49
simulate_aloha(CriteriaSanityCheckProblem1c_7())

0.5143570536828964